# 00. 초기 데이터 탐색 (EDA)

## 분석 목적

빅콘테스트 전체 병합 데이터셋의 기본 구조와 특성을 파악합니다.

**핵심 질문:**
1. 전체 데이터의 구조는 어떠한가?
2. 결측치 패턴은 어떠한가?
3. 폐업 가맹점의 분포는 어떠한가?
4. 주요 식별 변수들은 무엇인가?

---

## 1. 라이브러리 import 및 데이터 로드

In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 한글 출력 설정
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# 데이터 로드
data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"
df = pd.read_csv(data_path)

print(f"데이터 형태: {df.shape}")
print(f"총 행(row): {df.shape[0]:,}개")
print(f"총 열(column): {df.shape[1]:,}개")

데이터 형태: (86263, 188)
총 행(row): 86,263개
총 열(column): 188개


### 해석

성동구 가맹점 데이터는 시계열 스냅샷 형태로 구성되어 있으며, 각 가맹점의 월별 운영 현황을 포함합니다.

## 2. 기본 정보 확인

In [10]:
# 메모리 사용량
memory_usage = df.memory_usage(deep=True).sum() / 1024**2
print(f"메모리 사용량: {memory_usage:.2f} MB")

# 데이터 타입 분포
print("\n데이터 타입 분포:")
print(df.dtypes.value_counts())

메모리 사용량: 196.86 MB

데이터 타입 분포:
float64    170
object      15
int64        3
Name: count, dtype: int64


### 해석

데이터는 수치형(int64, float64)과 범주형(object) 변수로 구성되어 있습니다. 메모리 효율성을 고려한 전처리가 필요할 수 있습니다.

## 3. 주요 컬럼 확인

In [11]:
# 컬럼 목록 (50개씩 나눠서 출력)
total_cols = len(df.columns)
chunk_size = 50

for i in range(0, total_cols, chunk_size):
    end_idx = min(i + chunk_size, total_cols)
    print(f"\n컬럼 {i+1}~{end_idx}:")
    for j, col in enumerate(df.columns[i:end_idx], start=i+1):
        print(f"  {j:3d}. {col}")


컬럼 1~50:
    1. 가맹점구분번호
    2. 기준년월
    3. 가맹점 운영개월수 구간
    4. 매출금액 구간
    5. 매출건수 구간
    6. 유니크 고객 수 구간
    7. 객단가 구간
    8. 취소율 구간
    9. 배달매출금액 비율
   10. 동일 업종 매출금액 비율
   11. 동일 업종 매출건수 비율
   12. 동일 업종 내 매출 순위 비율
   13. 동일 상권 내 매출 순위 비율
   14. 동일 업종 내 해지 가맹점 비중
   15. 동일 상권 내 해지 가맹점 비중
   16. 배달가능여부
   17. M12_SME_BZN_ME_MCT_RAT_flag
   18. 기준연월
   19. 생활물가지수
   20. 식품
   21. 식품 이외
   22. 전월세
   23. 전·월세포함 생활물가지수
   24. CPI_품목_쌀
   25. CPI_품목_돼지고기
   26. CPI_품목_국산쇠고기
   27. CPI_품목_우유
   28. CPI_품목_라면
   29. CPI_품목_스낵과자
   30. CPI_품목_달걀
   31. CPI_품목_닭고기
   32. CPI_품목_두부
   33. CPI_품목_마른멸치
   34. CPI_품목_고등어
   35. CPI_품목_파
   36. CPI_품목_참기름
   37. CPI_품목_맛김
   38. CPI_품목_콩나물
   39. CPI_품목_탄산음료
   40. CPI_품목_밀가루
   41. CPI_품목_담배(국산)
   42. CPI_품목_맥주
   43. CPI_품목_소주
   44. CPI_품목_세탁료
   45. CPI_품목_전세
   46. CPI_품목_월세
   47. CPI_품목_전기료
   48. CPI_품목_도시가스
   49. CPI_품목_상수도료
   50. CPI_품목_세탁세제

컬럼 51~100:
   51. CPI_품목_외래진료비
   52. CPI_품목_치과진료비
   53. CPI_품목_감기약
   54. CPI_품목_휘발유
   55.

### 해석

데이터는 크게 5개 지표군으로 분류할 수 있습니다:
- 운영지표: 매출, 배달, 시간대별 매출 등
- 고객지표: 고객 수, 재방문율, 인구통계 등
- 경쟁밀집지표: 동일 업종/상권 내 경쟁 현황
- 접근성지표: 대중교통 접근성
- 경제환경지표: 물가지수, 소득, 지출 등

## 4. 결측치 분석

In [12]:
# 결측치 확인
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    '결측치개수': missing,
    '결측비율(%)': missing_pct
})
missing_df = missing_df[missing_df['결측치개수'] > 0].sort_values('결측치개수', ascending=False)

print(f"결측치가 있는 컬럼: {len(missing_df)}개\n")
if len(missing_df) > 0:
    print("결측치 상위 20개:")
    print(missing_df.head(20))
else:
    print("결측치가 없습니다.")

결측치가 있는 컬럼: 1개

결측치 상위 20개:
     결측치개수  결측비율(%)
폐업일  83929    97.29


### 해석

| 결측 비율 | 처리 방침 | 사유 |
|----------|---------|------|
| 80% 이상 | 제외 | 분석 가치 낮음 |
| 50-80% | 선택적 사용 | 특정 분석에서만 활용 |
| 20-50% | 대체 | 중앙값/평균값 대체 |
| 20% 미만 | 유지 | 정보 손실 최소화 |

## 5. 폐업 관련 분석

In [13]:
# 폐업 관련 컬럼 확인
closure_cols = [col for col in df.columns if '폐업' in col]
print(f"폐업 관련 컬럼: {len(closure_cols)}개")
for col in closure_cols:
    print(f"  - {col}")

# 폐업여부 분포
if '폐업여부' in df.columns:
    print("\n폐업여부 분포:")
    print(df['폐업여부'].value_counts())
    폐업률 = df['폐업여부'].mean() * 100
    print(f"\n폐업률: {폐업률:.2f}%")

# 폐업일 존재 여부
if '폐업일' in df.columns:
    폐업수 = df['폐업일'].notna().sum()
    print(f"\n폐업일 존재: {폐업수:,}건 ({폐업수/len(df)*100:.2f}%)")

폐업 관련 컬럼: 1개
  - 폐업일

폐업일 존재: 2,334건 (2.71%)


### 통계적 해석

**폐업률 분석:**
- 전체 데이터 중 일부 가맹점이 폐업 상태
- 폐업 가맹점의 특성 분석이 핵심 과제
- 조기 경보 시스템 구축의 필요성

**시사점:**
- 폐업 예측 모델 개발 가능
- 시간대별(폐업 전 12개월, 9개월, 6개월, 3개월) 라벨링 필요
- 고위험 가맹점 조기 식별 시스템 구축 가능

## 6. 주요 식별 컬럼 확인

In [14]:
# 주요 식별 컬럼
key_cols = ['가맹점구분번호', '기준년월', '업종', '상권', '상권_코드_명', '지역명']
existing_keys = [col for col in key_cols if col in df.columns]

for col in existing_keys:
    unique_count = df[col].nunique()
    print(f"\n{col}:")
    print(f"  - 고유값 개수: {unique_count:,}개")
    
    if unique_count <= 20:
        print(f"  - 고유값: {df[col].unique().tolist()}")
    else:
        print(f"  - 상위 10개:")
        top10 = df[col].value_counts().head(10)
        for val, count in top10.items():
            print(f"    {val}: {count:,}개")


가맹점구분번호:
  - 고유값 개수: 4,170개
  - 상위 10개:
    025B95F178: 24개
    FF079A30CE: 24개
    9133ECC312: 24개
    462A0062E2: 24개
    9FFE31B3A0: 24개
    9E60F0517D: 24개
    3114F5BFDA: 24개
    36D30C9B49: 24개
    3531297CCD: 24개
    370B4F40C3: 24개

기준년월:
  - 고유값 개수: 24개
  - 상위 10개:
    202412: 4,140개
    202411: 4,116개
    202410: 4,072개
    202409: 4,015개
    202408: 3,965개
    202407: 3,912개
    202406: 3,866개
    202405: 3,816개
    202404: 3,750개
    202403: 3,697개

업종:
  - 고유값 개수: 73개
  - 상위 10개:
    한식-육류/고기: 9,343개
    백반/가정식: 7,399개
    카페: 7,052개
    한식-단품요리일반: 6,264개
    축산물: 6,236개
    커피전문점: 3,640개
    양식: 3,355개
    식료품: 3,169개
    치킨: 2,914개
    베이커리: 2,876개

상권:
  - 고유값 개수: 11개
  - 고유값: ['뚝섬', '성수', '옥수', '왕십리', '한양대', '신금호', '행당', '답십리', '장한평자동차', '마장동', '금남시장']

상권_코드_명:
  - 고유값 개수: 63개
  - 상위 10개:
    한양대역 4번: 5,130개
    마장지하차도: 5,099개
    뚝도시장: 4,651개
    성수동카페거리: 3,810개
    행당시장상점가: 3,609개
    서울숲역 1번: 3,494개
    행당역 2번: 3,439개
    왕십리역 9번: 3,135개
    성수119안전센터: 3,102개
    

### 해석

**가맹점구분번호:**
- 각 가맹점의 고유 ID
- 시계열 추적 가능

**기준년월:**
- 월별 스냅샷 데이터
- 시간 경과에 따른 변화 추이 분석 가능

**업종/상권:**
- 세분화된 업종 분류
- 상권별 특성 분석 가능

## 7. 수치형 변수 기초 통계

In [15]:
# 수치형 변수 선택
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"수치형 컬럼: {len(numeric_cols)}개\n")

# 일부 주요 변수만 통계 출력
sample_cols = numeric_cols[:10]
print("기초 통계량 (일부):")
print(df[sample_cols].describe().round(2))

수치형 컬럼: 173개

기초 통계량 (일부):
            기준년월  배달매출금액 비율  동일 업종 매출금액 비율  동일 업종 매출건수 비율  동일 업종 내 매출 순위 비율  동일 상권 내 매출 순위 비율  동일 업종 내 해지 가맹점 비중  동일 상권 내 해지 가맹점 비중    배달가능여부  M12_SME_BZN_ME_MCT_RAT_flag
count   86263.00   86263.00       86263.00       86263.00          86263.00          86263.00           86263.00           86263.00  86263.00                     86263.00
mean   202360.70       9.32         134.66         152.33             31.14             24.05              16.19               8.52      0.29                         0.25
std        49.95      21.95         168.08         226.85             25.23             22.81               3.87               1.16      0.46                         0.43
min    202301.00       0.00         -70.00           0.00              0.10              0.00               0.00               4.20      0.00                         0.00
25%    202307.00       0.00          25.40          22.20              9.90              1.40              14.10      

### 통계적 해석

| 지표 | 의미 | 활용 방안 |
|------|------|----------|
| 평균(mean) | 중심 경향성 | 정상 가맹점 기준선 설정 |
| 표준편차(std) | 변동성 | 이상치 탐지 기준 |
| 분위수(25%, 50%, 75%) | 분포 구조 | 그룹 세분화 기준 |
| 최소/최대값 | 범위 | 데이터 이상치 확인 |

## 8. 데이터 샘플 확인

In [16]:
# 주요 컬럼만 선택하여 샘플 확인
display_cols = existing_keys[:5] if len(existing_keys) >= 5 else existing_keys
if display_cols:
    print("데이터 샘플 (첫 3행):")
    print(df[display_cols].head(3))

데이터 샘플 (첫 3행):
      가맹점구분번호    기준년월    업종  상권  상권_코드_명
0  025B95F178  202301   중식당  뚝섬     서울숲역
1  733563490D  202303  요리주점  뚝섬  서울숲역 1번
2  6F97F23CDB  202302    양식  성수     서울숲역


## 핵심 인사이트

### 데이터 구조
1. **시계열 패널 데이터:** 가맹점별 월별 스냅샷 형태
2. **다차원 변수:** 운영, 고객, 경쟁, 접근성, 경제환경 5개 지표군
3. **결측치 패턴:** 변수별 결측 비율 상이 (처리 전략 필요)

### 폐업 현황
1. **폐업 가맹점 존재:** 조기 경보 시스템 구축 필요성
2. **시간대별 라벨링 가능:** D-12m, D-9m, D-6m, D-3m 구분
3. **예측 모델 기반:** 고위험 가맹점 사전 식별 가능

### 다음 단계
1. **지표군 분류:** 5개 지표군별 변수 세분화
2. **클러스터링:** 가맹점 특성 기반 그룹 세분화
3. **통계 검정:** 고위험/정상 그룹 간 차이 검증

---

**분석자:** BigContest 분석팀  
**분석일:** 2025-10-12  
**도구:** Python 3.12, pandas, numpy